<a href="https://colab.research.google.com/github/Fred-1280/ChatbotPythonAi/blob/main/EmbeddingTeste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install -u -q google-generativeai

In [24]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('SECREAT_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

In [ ]:
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Titulo: A próxima geração de IA para desenvolvedores e Google Workspace"
  "\n"
  "Artigo completo:\n"
  "\n"
  "Gemini API & Google AI Studio: uma mandeira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content = sample_text,
                                 title = title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

In [67]:
DOCUMENT1 = {
    "Titulo": "David Bowie",
    "Conteudo": "David Bowie, nome artístico de David Robert Jones (Londres, 8 de janeiro de 1947 — Nova Iorque, 10 de janeiro de 2016)  foi um cantor, compositor, ator e produtor musical britânico."
}

DOCUMENT2 = {
    "Titulo": "Freddie Mercury",
    "Conteudo": "(nascido Farrokh Bulsara; Cidade de Pedra, 5 de setembro de 1946 — Londres, 24 de novembro de 1991) oi um cantor, pianista e compositor britânico, conhecido pelo seu trabalho com a banda britânica de rock Queen, que integrou como vocalista de 1970 até o ano de sua morte, 1991. É considerado como um dos melhores cantores de todos os tempos."
}

DOCUMENT3 = {
    "Titulo": "John Lennon",
    "Conteudo": "John Winston Ono Lennon (Liverpool, 9 de outubro de 1940 – Nova Iorque, 8 de dezembro de 1980) foi um cantor, compositor e ativista da paz britânico que fundou os Beatles, a banda de maior sucesso comercial na história da música popular. Sua parceria de composição com o colega de banda Paul McCartney foi uma das mais célebres da história da música."
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [ ]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

In [69]:
model = "models/embedding-001"

In [70]:
def emed_fn(title, text):
  return genai.embed_content(model=model,
                                 content = text,
                                 title = title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [ ]:
df["Embeddings"] = df.apply(lambda row: emed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

In [72]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content = consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [ ]:
consulta = 'Londres, 8 de janeiro de 1947'

trecho = gerar_e_buscar_consulta(consulta, df, model)

print(trecho)